In [1]:
import pymc as pm
import arviz as az
import numpy as np

WARNING (pytensor.configdefaults): g++ not detected!  PyTensor will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set PyTensor flags cxx to an empty string.
WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
print('Pymc3 version:', pm.__version__)
print('Arviz version:', az.__version__)
print('Numpy version:', np.__version__)

Pymc3 version: 5.10.4
Arviz version: 0.17.0
Numpy version: 1.26.4


### Zadanie 1

Hipotezy:

- $H_1$ główny podejrzany zabił,
- $H_2$ główny podejrzany nie zabił, 

Fakty:

- $E_1$ na miejscu zbrodni znaleziono odciski palców głównego podejrzanego,
- $E_2$ główny podejrzany nie ma alibi na czas popełnienia zabójstwa,
- $E_3$ główny podejrzany miał motyw zabicia ofiary,
- $E_4$ główny podejrzany był widziany w sądziedztwie miejsca, w którym mieszka nielegalny handlarz bronią,
- $E_5$ świadek zbrodni podał rysopis zabójcy nie pasujący do głównego podejrzanego. 

Prawdopodobieństwa:

$P(E_1|H_1)=0.7,\qquad P(E_1|H_2)=0.3,$

$P(E_2|H_1)=0.8,\qquad P(E_2|H_2)=0.4,$

$P(E_3|H_1)=0.9,\qquad P(E_3|H_2)=0.5,$

$P(E_4|H_1)=0.4,\qquad P(E_4|H_2)=0.2,$

$P(E_5|H_1)=0.2,\qquad P(E_5|H_2)=0.4.$ 

__W którym przypadku prawdopodobieństwo popełnienia zabójstwa byłoby największe?__

1. Gdyby znaleziono na miejscu zbrodni jego odciski palców.
2. Gdyby stwierdzono, że nie miał alibi i miał motyw.
3. Gdyby znaleziono na miejscu zbrodni jego odciski palców oraz stwierdzono, że był widziany w sąsiedztwie miejsca, w którym mieszka nielegalny handlarz bronią, ale świadek zbrodni podał rysopis zabójcy nie pasujący do głównego podejrzanego.

In [3]:
with pm.Model() as zabojstwo_model:
    # zmienna zabił o rozkładzie Bernoulliego z prawd. 0.5,
    # z prawd. 0.5 True (zabił), z prawd. 0.5 False (nie zabił)
    zabil = pm.Bernoulli('zabil', 0.5)

    # prawd. znalezienia odcisków palcow pod warunkiem że zabił
    # jeżeli zabił to p_odciski=0.7, jeżeli nie zabił to p_odciski=0.3
    p_odciski = pm.Deterministic('p_odciski', pm.math.switch(zabil, 0.7, 0.3))
    # zmienna odciski o rozkładie Bernoulliego
    odciski = pm.Bernoulli('odciski', p_odciski)

    # prawd. braku alibi podejrzanego pod warunkiem że zabił
    # jeżeli zabił to p_brak_alibi=0.8, jeżeli nie zabił to p_brak_alibi=0.4
    p_brak_alibi = pm.Deterministic('p_brak_alibi', pm.math.switch(zabil, 0.8, 0.4))
    # zmienna brak_alibi o rozkładie Bernoulliego
    brak_alibi = pm.Bernoulli('brak_alibi', p_brak_alibi)

    # prawd. posiadania motywu pod warunkiem że zabił
    # jeżeli zabił to p_motyw=0.9, jeżeli nie zabił to p_motyw=0.5
    p_motyw = pm.Deterministic('p_motyw', pm.math.switch(zabil, 0.9, 0.5))
    # zmienna motyw o rozkładie Bernoulliego
    motyw = pm.Bernoulli('motyw', p_motyw)

    # prawd. bycia widzianym obok domu handlarza bronia pod warunkiem że zabił
    # jeżeli zabił to p_obok_handlarza=0.4, jeżeli nie zabił to p_obok_handlarza=0.2
    p_obok_handlarza = pm.Deterministic('p_obok_handlarza', pm.math.switch(zabil, 0.4, 0.2))
    # zmienna obok_handlarza o rozkładie Bernoulliego
    obok_handlarza = pm.Bernoulli('obok_handlarza', p_obok_handlarza)

    # prawd. podania niepoprawnego rysopisu pod warunkiem że zabił
    # jeżeli zabił to p_niepoprawny_rysopis=0.2, jeżeli nie zabił to p_niepoprawny_rysopis=0.4
    p_niepoprawny_rysopis = pm.Deterministic('p_niepoprawny_rysopis', pm.math.switch(zabil, 0.2, 0.4))
    # zmienna niepoprawny_rysopis o rozkładie Bernoulliego
    niepoprawny_rysopis = pm.Bernoulli('niepoprawny_rysopis', p_niepoprawny_rysopis)

In [4]:
with zabojstwo_model:
    trace = pm.sample(10000, chains=1, return_inferencedata=True)

Sequential sampling (1 chains in 1 job)
BinaryGibbsMetropolis: [zabil, odciski, brak_alibi, motyw, obok_handlarza, niepoprawny_rysopis]


Sampling 1 chain for 1_000 tune and 10_000 draw iterations (1_000 + 10_000 draws total) took 15 seconds.
Only one chain was sampled, this makes it impossible to run some convergence checks


In [5]:
# prawd. że podejrzany zabił pod warunkiem znalezienia na miejscu zbrodni jego odcisków palców
p_zabil_odciski = (trace.posterior['zabil'].values[0]*trace.posterior['odciski'].values[0]).sum()/trace.posterior['odciski'].values[0].sum()
print('p_zabil_odciski:', p_zabil_odciski)

# prawd. że podejrzany zabił pod warunkiem że nie miał alibi i miał motyw
p_zabil_brak_alibi_motyw = (trace.posterior['zabil'].values[0]*trace.posterior['brak_alibi'].values[0]*trace.posterior['motyw'].values[0]).sum()/(trace.posterior['brak_alibi'].values[0]*trace.posterior['motyw'].values[0]).sum()
print('p_zabil_brak_alibi_motyw:', p_zabil_brak_alibi_motyw)

# prawd. że podejrzany zabił pod warunkiem że znaleziono odciski, był widziany, rysopis nie pasował
p_zabil_odciski_obok_handlarza_niepoprawny_rysopis = (trace.posterior['zabil'].values[0]*trace.posterior['odciski'].values[0]*trace.posterior['obok_handlarza'].values[0]*trace.posterior['niepoprawny_rysopis'].values[0]).sum()/(trace.posterior['odciski'].values[0]*trace.posterior['obok_handlarza'].values[0]*trace.posterior['niepoprawny_rysopis'].values[0]).sum()
print('p_zabil_brak_alibi_motyw:', p_zabil_odciski_obok_handlarza_niepoprawny_rysopis)

p_zabil_odciski: 0.6881099382593109
p_zabil_brak_alibi_motyw: 0.7892770819600088
p_zabil_brak_alibi_motyw: 0.7075718015665796


### Zadanie 2

Prawdopodobieństwo włamania: $P(B) = 0.001$

Prawdopodobieństwo trzęsienia ziemi: $P(E) = 0.002$

Prawdopodobieństwo uruchomienia się alarmu $P(A)$:
| B | E | P(A\|B,E)|
| - | - | - |
| T | T |  0.95 |
| T | F |  0.94 |
| F | T |  0.29 |
| F | F | 0.001 |

Prawdopodobieństwo, że John zadzwoni $P(J)$:
| A | P(J\|A) |
| - | - |
| T | 0.90 |
| F | 0.05 |

Prawdopodobieństwo, że Mary zadzwoni $P(M)$:
| A | P(M\|A) |
| - | - |
| T | 0.70 |
| F | 0.01 |

__Jakie jest prawdopodobieństwo, że:__
1. włączy się alarm?
2. doszło do włamanie jeśli wiadom, że włączył się alarm?
3. zdarzyło się trzęsienie ziemi jeśli wiadomo, żę włączył się alarm?
1. w razie włamania ktoś zadzwoni?
2. zawiadomienie o włamaniu jest fałszywe?
3. rozległ się alarm, przy czym nie wystąpiło ani trzęsienie ziemi ani włamanie, ale oboje John i Mary zadzwonili? (prawd. bezwarunkowe)

In [6]:
with pm.Model() as model:
    burglary = pm.Bernoulli('burglary', 0.001)
    earthquake = pm.Bernoulli('earthquake', 0.002)
    alarm_p = pm.Deterministic('alarm_p', pm.math.switch(earthquake, pm.math.switch(burglary, 0.95, 0.29), pm.math.switch(burglary, 0.94, 0.001)))
    alarm = pm.Bernoulli('alarm', alarm_p)
    johnCalls_p = pm.Deterministic('johnCalls_p', pm.math.switch(alarm, 0.90, 0.05))
    johnCalls = pm.Bernoulli('johnCalls', johnCalls_p)
    maryCalls_p = pm.Deterministic('maryCalls_p', pm.math.switch(alarm, 0.70, 0.01))
    maryCalls = pm.Bernoulli('maryCalls', maryCalls_p)

    trace = pm.sample(1000000, chains=1, return_inferencedata=True)

Sequential sampling (1 chains in 1 job)
BinaryGibbsMetropolis: [burglary, earthquake, alarm, johnCalls, maryCalls]


Sampling 1 chain for 1_000 tune and 1_000_000 draw iterations (1_000 + 1_000_000 draws total) took 877 seconds.
Only one chain was sampled, this makes it impossible to run some convergence checks


In [7]:
#Jakie jest prawdopodobieństwo, że:

#włączy się alarm
p_alarm = trace.posterior['alarm'].values[0].sum()/len(trace.posterior['alarm'].values[0])
print('p_alarm:', p_alarm)

#doszło do włamania jeśli wiadomo, że włączył się alarm
p_burglary_alarm = (trace.posterior['burglary'].values[0]*trace.posterior['alarm'].values[0]).sum()/trace.posterior['alarm'].values[0].sum()
print('p_burglary_alarm:', p_burglary_alarm)

#zdarzyło się trzęsienie ziemi jeśli wiadomo, żę włączył się alarm
p_earthquake_alarm = (trace.posterior['earthquake'].values[0]*trace.posterior['alarm'].values[0]).sum()/trace.posterior['alarm'].values[0].sum()
print('p_earthquake_alarm:', p_earthquake_alarm)

#w razie włamania ktoś zadzwoni
p_calls_bulglary = (np.logical_not(np.logical_not(trace.posterior['johnCalls'].values[0])*np.logical_not(trace.posterior['maryCalls'].values[0]))*trace.posterior['burglary'].values[0]).sum()/(trace.posterior['burglary'].values[0].sum())
print('p_calls_bulglary:', p_calls_bulglary)

#zawiadomienie o włamaniu jest fałszywe
p_calls_notBurglary = (np.logical_not(np.logical_not(trace.posterior['johnCalls'].values[0])*np.logical_not(trace.posterior['maryCalls'].values[0]))*np.logical_not(trace.posterior['burglary'].values[0])).sum()/(np.logical_not(np.logical_not(trace.posterior['johnCalls'].values[0])*np.logical_not(trace.posterior['maryCalls'].values[0])).sum())
print('p_calls_notBurglary:', p_calls_notBurglary)

#rozległ się alarm, przy czym nie wystąpiło ani trzęsienie ziemi ani włamanie, ale oboje John i Mary zadzwonili
p_alarm_notEarthquake_notBurglary_calls = (trace.posterior['alarm'].values[0]*np.logical_not(trace.posterior['earthquake'].values[0])*np.logical_not(trace.posterior['burglary'].values[0])*trace.posterior['maryCalls'].values[0]*trace.posterior['johnCalls'].values[0]).sum()/len(trace.posterior['alarm'].values[0])
print('p_alarm_notEarthquake_notBurglary_calls:', p_alarm_notEarthquake_notBurglary_calls)

p_alarm: 0.00259
p_burglary_alarm: 0.37335907335907337
p_earthquake_alarm: 0.23938223938223938
p_calls_bulglary: 0.9127906976744186
p_calls_notBurglary: 0.984818449934729
p_alarm_notEarthquake_notBurglary_calls: 0.000636
